<span style="font-size:24px; font-family:'Roboto'; font-weight:bold;">
Script to PoP iMOD Sim HD outputs
</span>

You're advised to run the notebook cells one by one to understand what they're doing and avoid any problems.

In [1]:
from WS_Mdl import utils as U
from WS_Mdl import utils_imod as UIM
from WS_Mdl import geo as G
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# Options

In [2]:
MdlN_S = 'NBr13'
SmB = True

# Infer parameters

In [4]:
d_paths = U.get_MdlN_paths(MdlN_S)
d_INI = U.INI_to_d(d_paths['path_INI'])
MdlN_B, path_PoP, path_MdlN_S, path_MdlN_B, Mdl, path_Out_HD, path_Out_HD_B = [ d_paths[v] for v in ['MdlN_B', 'path_PoP', 'path_MdlN', 'path_MdlN_B', 'Mdl', 'path_Out_HD', 'path_Out_HD_B'] ]

✅ - NBr13 paths extracted from RunLog and returned as dictionary with keys:
Mdl, MdlN_B, path_Mdl, path_MdlN, path_Out_HD, path_PoP, path_PoP_Out_MdlN, path_INI, path_BAT, path_PRJ, path_Smk, path_MdlN_B, path_Out_HD_B, path_PoP_Out_MdlN_B, path_INI_B, path_BAT_B, path_PRJ_B, path_Smk_B
✅ - INI file C:/OD/WS_Mdl\models/NBr\code/Mdl_Prep/Mdl_Prep_NBr13.ini read successfully. Dictionary created with 21 keys.


In [5]:
l_Mo_winter = [10, 11, 12, 1, 2, 3]
l_Mo_summer = [i for i in range(1,13) if i not in l_Mo_winter]
l_layers = ['L1']

In [6]:
Se_path_HD_B = pd.Series([i for i in os.listdir(path_Out_HD_B) if i.lower().endswith('.idf')]).apply(lambda x: os.path.join(path_Out_HD_B, x))
Se_Fi_HD_B = Se_path_HD_B.apply(lambda x: os.path.basename(x))
Se_path_HD_S = pd.Series([i for i in os.listdir(path_Out_HD) if i.lower().endswith('.idf')]).apply(lambda x: os.path.join(path_Out_HD, x))
Se_Fi_HD_S = Se_path_HD_S.apply(lambda x: os.path.basename(x))

In [7]:
Se_Fi_B_Mo = Se_Fi_HD_B.str.split("_").str[1].str[4:6].astype(int)
Se_Fi_S_Mo = Se_Fi_HD_S.str.split("_").str[1].str[4:6].astype(int)

# Load IDF's and calculate AVGs

## B

In [9]:
DF_IDF_B = UIM.IDFs_to_DF(Se_path_HD_B.iloc[:10])

Loading .IDF files: 100%|██████████| 10/10 [00:00<00:00, 542.52it/s]


In [10]:
DF_IDF_B

,path,IDF
0,C:/OD/WS_Mdl\models/NBr\Sim/NBr12\GWF_1/MODELO...,"([[2.1205072, 2.11753, 2.0634706, 1.9590068, 1..."
1,C:/OD/WS_Mdl\models/NBr\Sim/NBr12\GWF_1/MODELO...,"([[2.0031214, 1.992437, 1.9591572, 1.9046625, ..."
2,C:/OD/WS_Mdl\models/NBr\Sim/NBr12\GWF_1/MODELO...,"([[1.9608903, 1.9484155, 1.9224886, 1.8844573,..."
3,C:/OD/WS_Mdl\models/NBr\Sim/NBr12\GWF_1/MODELO...,"([[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
4,C:/OD/WS_Mdl\models/NBr\Sim/NBr12\GWF_1/MODELO...,"([[1.9605143, 1.9480246, 1.9221693, 1.8842961,..."
5,C:/OD/WS_Mdl\models/NBr\Sim/NBr12\GWF_1/MODELO...,"([[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
6,C:/OD/WS_Mdl\models/NBr\Sim/NBr12\GWF_1/MODELO...,"([[1.9603072, 1.9478096, 1.9219983, 1.8842221,..."
7,C:/OD/WS_Mdl\models/NBr\Sim/NBr12\GWF_1/MODELO...,"([[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
8,C:/OD/WS_Mdl\models/NBr\Sim/NBr12\GWF_1/MODELO...,"([[1.9594263, 1.9468943, 1.9213105, 1.8840382,..."
9,C:/OD/WS_Mdl\models/NBr\Sim/NBr12\GWF_1/MODELO...,"([[2.4211159, 2.4148133, 2.4003341, 2.3783822,..."


In [ ]:
S_path_HD_B.iloc[0]

In [ ]:
print([os.path.basename(i) for i in S_path_HD_B.iloc[0:10]])

In [ ]:
print([os.path.basename(i) for i in S_path_HD_B.iloc[-10:]])

In [ ]:
XA_HD_B = imod.formats.idf.open(list(S_path_HD_B), pattern="{name}_{DT}_L{layer}")

In [ ]:
XA_HD_B

In [ ]:
XA_HD_B.coords["dt"] = pd.to_datetime(XA_HD_B.coords["dt"].values, format="%Y%m%d%H%M%S")

In [ ]:
XA_HD_B.coords["month"] = XA_HD_B.coords["dt"].to_index().to_period("M")

In [ ]:
XA_HD_B

---

In [ ]:
A_B_AVG = np.mean(np.stack(DF_IDF_B['IDF'].apply(lambda x: x[0])), axis=0)
A_B_AVG_summer = np.mean(np.stack(DF_IDF_B.loc[ S_Fi_B_Mo.isin(l_Mo_summer), 'IDF'].apply(lambda x: x[0])), axis=0)
A_B_AVG_winter = np.mean(np.stack(DF_IDF_B.loc[ S_Fi_B_Mo.isin(l_Mo_winter), 'IDF'].apply(lambda x: x[0])), axis=0)
d_A_B_AVG = {'full_year': A_B_AVG,
             'summer': A_B_AVG_summer,
             'winter': A_B_AVG_winter}

In [ ]:
IDF_MtDt = DF_IDF_B.at[0, 'IDF'][1] # The metadata for all IDF's is the same, except for time. But time doesn't matter for what we'll do.

In [ ]:
for k in d_A_B_AVG:
    group = '' if k=='full_year' else (k+'_')
    path_Out = os.path.join(path_PoP, 'HD_map', f'{Mdl}{SimN_B}', f'HD_{group}{Mdl}{SimN_B}.')
    os.makedirs(os.path.dirname(path_Out), exist_ok=True)
    A_to_Raster_n_IDF(d_A_B_AVG[k], IDF_MtDt, path_Out)

## S

In [ ]:
DF_IDF_S = make_DF_IDF(path_S, S_Fi_S)

In [ ]:
A_S_AVG = np.mean(np.stack(DF_IDF_S['IDF'].apply(lambda x: x[0])), axis=0)
A_S_AVG_summer = np.mean(np.stack(DF_IDF_S.loc[ S_Fi_S_Mo.isin(l_Mo_summer), 'IDF'].apply(lambda x: x[0])), axis=0)
A_S_AVG_winter = np.mean(np.stack(DF_IDF_S.loc[ S_Fi_S_Mo.isin(l_Mo_winter), 'IDF'].apply(lambda x: x[0])), axis=0)
d_A_S_AVG = {'full_year': A_S_AVG,
             'summer': A_S_AVG_summer,
             'winter': A_S_AVG_winter}

In [ ]:
IDF_MtDt = DF_IDF_S.at[0, 'IDF'][1] # The metadata for all IDF's is the same, except for time. But time doesn't matter for what we'll do.

In [ ]:
for k in d_A_S_AVG:
    group = '' if k=='full_year' else (k+'_')
    path_Out = os.path.join(path_PoP, 'HD_map', f'{Mdl}{SimN_S}', f'HD_{group}{Mdl}{SimN_S}.')
    os.makedirs(os.path.dirname(path_Out), exist_ok=True)
    A_to_Raster_n_IDF(d_A_S_AVG[k], IDF_MtDt, path_Out)